In [ ]:
# My Encoder
encoder_model = keras.models.Sequential([
    keras.layers.Input(shape=(t_timesteps, m_chords), name='encoder_input'),  # Might flatten this?
    keras.layers.Dense(128, activation='tanh', name='encoder_hidden_layer1'),  # Densely connected to the previous layer
    keras.layers.Dense(latent_dim, activation='tanh', name='encoder_hidden_layer2'),   # Outputs encoded input
], name='encoder_model')

# My Decoder
decoder_model = keras.models.Sequential([
    keras.layers.Input(shape=(t_timesteps, latent_dim), name='latent_input'), 
    keras.layers.Dense(128, activation='tanh', name='decoder_hidden_layer1'),  # Densely connected to previous layer
    keras.layers.Dense(m_chords, activation='sigmoid', 
        name='decoder_output')
], name='decoder_model')

## My Stacked autoencoder
# For this network, input is (n_samples, t_timesteps, m_one_hot_digits)
# The length of the m_one_hot_digits dimension is determined by the number
# of uniquely encoded chords/notes catgeories encoded during parsing.
stacked_autoencoder = tf.keras.models.Sequential([
    keras.layers.Input(shape=(t_timesteps, m_chords), name='input'),
    encoder_model, 
    decoder_model
])

In [ ]:
# Compile the model
stacked_autoencoder.compile(
    loss='binary_crossentropy', 
    optimizer='rmsprop',  # Could use Adam instead?
    metrics=[keras.metrics.BinaryCrossentropy()]
)

In [ ]:
# Encoder figure
keras.utils.plot_model(encoder_model, to_file='./figures/enc.png', show_shapes=True, expand_nested=True)

In [ ]:
# Decoder figure
keras.utils.plot_model(decoder_model, to_file='./figures/dec.png', show_shapes=True, expand_nested=True)

In [ ]:
# Stacked autoencoder figure
keras.utils.plot_model(stacked_autoencoder, to_file='./figures/ae.png', show_shapes=True, expand_nested=True,)

In [ ]:
# Summarize
stacked_autoencoder.summary()

In [ ]:
# Fit the separated autoencoder model
fit_model_separated_autoencoder = False
if (fit_model_separated_autoencoder):
    history = stacked_autoencoder.fit(
        one_hot_train_chords_array, 
        one_hot_train_chords_array, 
        epochs=128,
        verbose=1,
        validation_data=[one_hot_test_chords_array, one_hot_test_chords_array]
        # callbacks=[
        #     TensorBoard(
        #         log_dir='logs\\model', 
        #         histogram_freq=1, 
        #         profile_batch=100000000
        #     ), 
        # ]   
    )